# Lambda phage

In [ ]:
from output_helper import *
from scripts.reference_solutions.ssa_helper import SSASol
%matplotlib inline

In [ ]:
with xr.open_dataset("output/lp/output_t1000.nc") as ds:
    grid = GridInfo(ds)
    lr_sol = LRSol(ds, grid)
    slice_vec = np.array([0, 0, 0, 0, 0])
    idx_2D = np.array([0, 1])
    P_marginal = lr_sol.marginalDistributions()
    P_sliced = lr_sol.slicedDistributions(slice_vec)
    P_marginal2D = lr_sol.marginalDistribution2D(idx_2D)

In [ ]:
result = np.load("scripts/reference_solutions/lp_ssa_ref.npy")
ssa_sol = SSASol(result)
P_full_ssa = ssa_sol.calculateFullDistribution()
P_marginal_ssa, P_marginal2D_ssa, P_sliced_ssa, P_sliced2D_ssa = ssa_sol.calculateObservables(slice_vec, idx_2D)

Additional parameters:

In [ ]:
idx = np.arange(5)
mesh = [grid.bin[j] * np.arange(n_el) + grid.liml[j] for j, n_el in enumerate(grid.n)]
mesh_ssa = [np.arange(el) + ssa_sol.n_min[j] for j, el in enumerate(ssa_sol.n)]

t = 10
time_step = 1
snapshot = 100

## Figure 1
One-dimensional marginal distributions for $t = 10$

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(6, 6))
plotP1Dmult(axs, [P_marginal, P_marginal_ssa[-1]], [mesh, mesh_ssa], ["DLR approx.", "SSA"], idx, "MD")
axs.flat[0].legend()
fig.tight_layout()
fig.delaxes(axs[-1, -1])
fig.savefig('plots/lp_fig1.pgf')

## Figure 2
Sliced distributions for $x_0 = (0,\,0,\,0,\,0,\,0)$, $t = 10$

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(6, 2))
plotP1Dmult(axs, [P_sliced, P_sliced_ssa[-1]], [mesh, mesh_ssa], ["DLR approx.", "SSA"], idx, "S")
axs.flat[0].legend()
plt.setp(axs.flat, yscale="log")
fig.tight_layout()
fig.savefig('plots/lp_fig2.pdf')

## Figure 3
Two-dimensional marginal distribution of the $S_1$-$S_2$ plane for $t = 10$

Reshape `P_marginal2D` and `P_marginal2D_ssa` such that they are on the same domain:

In [ ]:
n_min = np.array([max(grid.liml[i], ssa_sol.n_min[i])
                 for i in [0, 1]], dtype="int64")
n_max = np.array([min(grid.n[i] + grid.liml[i], ssa_sol.n_max[i])
                  for i in [0, 1]], dtype="int64")
n = n_max - n_min
Pred = np.zeros(n[:2])
Pred_ssa = np.zeros(n[:2])

liml_red = np.array([n_min[i] - grid.liml[i]
                     for i in [0, 1]], dtype="int64")
liml_red_ssa = np.array([n_min[i] - ssa_sol.n_min[i]
                         for i in [0, 1]], dtype="int64")

for i in range(n[0]):
    for j in range(n[1]):
        Pred[i, j] = P_marginal2D[i + liml_red[0], j + liml_red[1]]
        Pred_ssa[i, j] = P_marginal2D_ssa[-1][(i + liml_red_ssa[0]) + ssa_sol.n[0] * (j + liml_red_ssa[1])]

In [ ]:
xx1, xx2 = np.meshgrid(
    np.arange(n[0]) + n_min[0], np.arange(n[1]) + n_min[1], indexing='ij')
fig, axs = plt.subplots(1, 2, figsize=(4, 4))
plotP2D(axs, [Pred, Pred_ssa], [xx1, xx2], ["DLR approximation", "SSA"])
plt.setp(axs.flat, xlim=[0, 8], ylim=[0, 20])
fig.tight_layout()

Maximal difference for the two-dimensional marginal distribution:

In [ ]:
print(np.amax(np.abs(Pred - Pred_ssa)))